In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Dropout, ConvLSTM1D, Flatten, Conv2D, Input ,LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam

import tensorflow as tf
from sklearn.preprocessing import StandardScaler


In [2]:
datapath = 'https://raw.githubusercontent.com/JonathanC1/Datasets/master/TorNonTor/SelectedFeatures-10s-TOR-NonTOR.csv'
dataframe = pd.read_csv(datapath,low_memory=False)

In [3]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [4]:
print(dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print(list(dataframe))

(16088, 29)
        Source Port   Destination Port   Protocol   Flow Duration   
count  16088.000000       16088.000000    16088.0    1.608800e+04  \
mean   36296.304388       11777.729426        6.0    4.422426e+06   
std    21217.827170       19907.298714        0.0    4.650899e+06   
min       21.000000          21.000000        6.0    1.000000e+00   
25%    33224.000000         443.000000        6.0    3.710925e+04   
50%    41282.000000         443.000000        6.0    7.780915e+05   
75%    54420.000000        9001.000000        6.0    9.965477e+06   
max    65087.000000       65514.000000        6.0    1.000000e+07   

        Flow IAT Max   Flow IAT Min   Fwd IAT Max   Fwd IAT Min   Bwd IAT Max   
count   1.608800e+04   1.608800e+04  1.608800e+04  1.608800e+04  1.608800e+04  \
mean    8.139237e+05   6.711377e+04  7.562887e+05  8.203885e+04  5.994034e+05   
std     1.775624e+06   4.228532e+05  1.745859e+06  6.422426e+05  1.525304e+06   
min     1.000000e+00  -2.255000e+03  0.000

In [5]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

        Protocol   Flow Duration   Flow IAT Max   Flow IAT Min   Fwd IAT Max   
count    16088.0    16088.000000   16088.000000   16088.000000  16088.000000  \
mean         0.0        0.442243       0.081417       0.007002      0.075651   
std          0.0        0.465090       0.177617       0.042685      0.174636   
min          0.0        0.000000       0.000000       0.000000      0.000000   
25%          0.0        0.003711       0.003708       0.000228      0.000000   
50%          0.0        0.077809       0.010545       0.000233      0.006994   
75%          0.0        0.996548       0.038336       0.003619      0.036599   
max          0.0        1.000000       1.000000       1.000000      1.000000   

        Fwd IAT Min   Bwd IAT Max   Bwd IAT Min   Active Mean   Active Std   
count  16088.000000  1.608800e+04  16088.000000  16088.000000      16088.0  \
mean       0.008238  5.995891e-02      0.005802      0.005490          0.0   
std        0.064241  1.525777e-01      0.0581

In [6]:
change_labels = lambda x: 1 if x == 'nonTOR' else 0
y_normalised = dataframe['label'].apply(change_labels)

X_train, X_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size = 0.20)

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_train = np.asarray(X_train).astype(np.float32)
X_train.shape


(12870, 24, 1)

In [14]:
#definir el modelo
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(12, input_shape=(x_normalised.shape[1],1), activation='relu')) # 12 8 Son las neuronas
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.add(LSTM(128,activation='relu',input_shape=(x_normalised.shape[1],1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])

In [20]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

Epoch 1/50
403/403 [==============================] - 8s 19ms/step - loss: 0.0510 - acc: 0.9819 - val_loss: 0.0788 - val_acc: 0.9823
Epoch 2/50
403/403 [==============================] - 8s 19ms/step - loss: 0.0654 - acc: 0.9785 - val_loss: 0.0489 - val_acc: 0.9832
Epoch 3/50
403/403 [==============================] - 8s 19ms/step - loss: 0.0513 - acc: 0.9815 - val_loss: 0.0492 - val_acc: 0.9823
Epoch 4/50
403/403 [==============================] - 9s 23ms/step - loss: 0.0498 - acc: 0.9828 - val_loss: 0.0612 - val_acc: 0.9820
Epoch 5/50
403/403 [==============================] - 8s 19ms/step - loss: 0.0501 - acc: 0.9819 - val_loss: 0.0523 - val_acc: 0.9832
Epoch 6/50
403/403 [==============================] - 8s 20ms/step - loss: 0.0487 - acc: 0.9827 - val_loss: 0.0576 - val_acc: 0.9820
Epoch 7/50
403/403 [==============================] - 8s 19ms/step - loss: 0.0624 - acc: 0.9803 - val_loss: 0.0723 - val_acc: 0.9817
Epoch 8/50
403/403 [==============================] - 8s 19ms/step - 

In [21]:
# Evaluate the model
X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
score = model.evaluate(X_test, y_test, verbose=0)
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test accuracy: 0.9832193851470947
